In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
with open('model\LSTM_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [3]:
df = pd.read_csv('../data/consolidated_data/training_dataset.csv')
df.drop(columns=['Unnamed: 0','state'], inplace=True)
df = df.groupby('date').sum()
df.index = pd.to_datetime(df.index)
df = df.asfreq('d')

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(df)
df_scaled = scaler.transform(df)

In [5]:
split_index = int(0.5 * len(df))
train_data = df.iloc[:split_index]
test_data = df.iloc[split_index:]

In [6]:
trainX = []
past = 14
future = 1
for i in range(past, len(df_scaled) + 1):
    trainX.append(df_scaled[i-past:i,0:df.shape[1]])

In [7]:
trainX = np.array(trainX)

In [8]:
trainX

array([[[-0.58417768, -0.58489877, -0.64306305, ..., -0.49636467,
         -0.48878824, -0.5936777 ],
        [-0.58486359, -0.58489877, -0.64306305, ..., -0.49636467,
         -0.48878824, -0.59403614],
        [-0.58486359, -0.58489877, -0.64306305, ..., -0.49636467,
         -0.48878824, -0.59403614],
        ...,
        [-0.58452064, -0.58472579, -0.64295481, ..., -0.49636467,
         -0.48878824, -0.59343379],
        [-0.58452064, -0.58489877, -0.64292388, ..., -0.49636467,
         -0.48878824, -0.59368649],
        [-0.58469211, -0.58489877, -0.64290841, ..., -0.49636467,
         -0.48878824, -0.59394653]],

       [[-0.58486359, -0.58489877, -0.64306305, ..., -0.49636467,
         -0.48878824, -0.59403614],
        [-0.58486359, -0.58489877, -0.64306305, ..., -0.49636467,
         -0.48878824, -0.59403614],
        [-0.58486359, -0.58489877, -0.64306305, ..., -0.49636467,
         -0.48878824, -0.59403614],
        ...,
        [-0.58452064, -0.58489877, -0.64292388, ..., -

In [9]:
future_dates_count = 365//2
forecast_dates = pd.date_range(start=test_data.index[-1], periods=future_dates_count)
forecast = model.predict(trainX[-future_dates_count:])

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step


In [11]:
forecast_copies = np.repeat(forecast,df.shape[1],axis=-1)
pred = scaler.inverse_transform(forecast_copies)[:,0]

In [15]:
forecast_df = pd.DataFrame({'Date': forecast_dates, 'Forecast': pred})
forecast_df.set_index('Date', inplace=True)

In [16]:
forecast_df

,Forecast
Date,
2024-04-20,448.339844
2024-04-21,444.220703
2024-04-22,454.175537
2024-04-23,462.627686
2024-04-24,472.884277
...,...
2024-10-14,344.660156
2024-10-15,354.895020
2024-10-16,371.519775
